In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType
hist = [
    ("Alice", 34, "HR",1,"2001-01-01",None),
    ("Bob", 45, "Finance",1,"2001-01-01",None),
    ("Cathy", 29, "IT",1,"2001-01-01",None)
]
schema = StructType([
    StructField("Name", StringType(), True),
    StructField("Age", IntegerType(), True),
    StructField("Department", StringType(), True),
    StructField("is_active", IntegerType(), True),
    StructField("Start_date", StringType(), True),   # or DateType if you want proper dates
    StructField("End_date", DateType(), True)
])

df_hist = spark.createDataFrame(hist,schema)
df_hist.show()

+-----+---+----------+---------+----------+--------+
| Name|Age|Department|is_active|Start_date|End_date|
+-----+---+----------+---------+----------+--------+
|Alice| 34|        HR|        1|2001-01-01|    NULL|
|  Bob| 45|   Finance|        1|2001-01-01|    NULL|
|Cathy| 29|        IT|        1|2001-01-01|    NULL|
+-----+---+----------+---------+----------+--------+



In [0]:
inc_data=[
    ("Alice", 37, "HR"),
    ("britney",67,"IT")]

columns = ["Name", "Age", "Department"]
df_inc = spark.createDataFrame(inc_data, columns)
df_inc.show()

+-------+---+----------+
|   Name|Age|Department|
+-------+---+----------+
|  Alice| 37|        HR|
|britney| 67|        IT|
+-------+---+----------+



In [0]:
from pyspark.sql.functions import *
df_old=df_hist.join(df_inc,df_hist["name"]==df_inc["name"],'left_semi').withColumn("is_active",lit(0)).withColumn("end_date",current_date())
df_old.show()

+-----+---+----------+---------+----------+----------+
| Name|Age|Department|is_active|Start_date|  end_date|
+-----+---+----------+---------+----------+----------+
|Alice| 34|        HR|        0|2001-01-01|2025-12-10|
+-----+---+----------+---------+----------+----------+



In [0]:
df_update=df_inc.join(df_hist,df_hist["name"]==df_inc["name"],'left_semi').withColumn("is_active",lit(1)).withColumn("start_date",current_date()).withColumn("end_date",lit(None))
df_update.show()

+-----+---+----------+---------+----------+--------+
| Name|Age|Department|is_active|start_date|end_date|
+-----+---+----------+---------+----------+--------+
|Alice| 37|        HR|        1|2025-12-10|    NULL|
+-----+---+----------+---------+----------+--------+



In [0]:
df_o=df_hist.join(df_inc,df_hist["name"]==df_inc["name"],'left_anti')
df_o.show()

+-----+---+----------+---------+----------+--------+
| Name|Age|Department|is_active|Start_date|End_date|
+-----+---+----------+---------+----------+--------+
|  Bob| 45|   Finance|        1|2001-01-01|    NULL|
|Cathy| 29|        IT|        1|2001-01-01|    NULL|
+-----+---+----------+---------+----------+--------+



In [0]:
df_new=df_inc.join(df_hist,df_hist["name"]==df_inc["name"],'left_anti').withColumn("is_active",lit(1)).withColumn("start__date",current_date()).withColumn("end_date",lit(None))
df_new.show()

+-------+---+----------+---------+-----------+--------+
|   Name|Age|Department|is_active|start__date|end_date|
+-------+---+----------+---------+-----------+--------+
|britney| 67|        IT|        1| 2025-12-10|    NULL|
+-------+---+----------+---------+-----------+--------+



In [0]:
from functools import reduce
def unionall(*a):
    return reduce(DataFrame.unionAll,a)
df_final=unionall(df_old,df_update,df_o,df_new)
df_final.show()

+-------+---+----------+---------+----------+----------+
|   Name|Age|Department|is_active|Start_date|  end_date|
+-------+---+----------+---------+----------+----------+
|  Alice| 34|        HR|        0|2001-01-01|2025-12-10|
|  Alice| 37|        HR|        1|2025-12-10|      NULL|
|    Bob| 45|   Finance|        1|2001-01-01|      NULL|
|  Cathy| 29|        IT|        1|2001-01-01|      NULL|
|britney| 67|        IT|        1|2025-12-10|      NULL|
+-------+---+----------+---------+----------+----------+



In [0]:
 df_final.show()

+-------+---+----------+---------+----------+----------+
|   Name|Age|Department|is_active|Start_date|  end_date|
+-------+---+----------+---------+----------+----------+
|  Alice| 34|        HR|        0|2001-01-01|2025-12-10|
|  Alice| 37|        HR|        1|2025-12-10|      NULL|
|    Bob| 45|   Finance|        1|2001-01-01|      NULL|
|  Cathy| 29|        IT|        1|2001-01-01|      NULL|
|britney| 67|        IT|        1|2025-12-10|      NULL|
+-------+---+----------+---------+----------+----------+



In [0]:
trg=df.write.saveAsTable("scd2")

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-6655263193627650>, line 1
----> 1 trg=df.write.saveAsTable("scd2")

NameError: name 'df' is not defined

In [0]:
%sql
Select * from workspace.default.scd2